In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

In [14]:
df_qa = client.to_pandas("""

SELECT
  app.project_name,
  projects.display_name,
  projects.open_source_observer_slug,
  MAX(app.updated_at) AS updated_at,
  ARRAY_AGG(DISTINCT CASE WHEN pdl.artifact_url IS NOT NULL THEN pdl.artifact_url ELSE '' END) AS defillama,
  ARRAY_AGG(DISTINCT pr.artifact_url) AS github
FROM stg_op_atlas_application AS app
JOIN stg_op_atlas_project AS projects
  ON app.project_id = projects.project_id
LEFT JOIN stg_op_atlas_project_repository AS pr
  ON pr.project_id = app.project_id
LEFT JOIN stg_op_atlas_project_defillama AS pdl
  ON pdl.project_id = app.project_id
WHERE app.project_name IN (
  '0x808b31862cec6dccf3894bb0e76ce4ca298e4c2820e2ccbf4816da740463b220', -- OSO 'fractal-visions'
  '0xed3d54e5394dc3ed01f15a67fa6a70e203df31c928dad79f70e25cb84f6e2cf9', -- OSO 'host-it'
  '0x15b210abdc6acfd99e60255792b2b78714f4e8c92c4c5e91b898d48d046212a4', -- OSO 'defieye'
  '0xd4f0252e6ac2408099cd40d213fb6f42e8fa129b446d6e8da55e673598ef14c0', -- DL 'moonwell' (missing from app)
  '0xfd9f98de666c5b0a5ce96d4a4b8d4ceee9f8c2156734d57bf6c23d0cff183e90', -- DL 'kim-exchange' (missing from app)
  '0x96767e87a27cdb9798f40c3a6fd78e70da611afe53a5e45cbaafc50cae4ad0e7', -- DL 'sonus' (legacy parent protocol name)
  '0x7262ed9c020b3b41ac7ba405aab4ff37575f8b6f975ebed2e65554a08419f8f4', -- DL 'sablier-finance' (legacy parent protocol name)
  '0x9c691ba3012549259099205cc1a7ca4d09d9b7a24a1b7b821b52c7bf1c9b89f4', -- DL 'origin-defi' (legacy parent protocol name)
  '0xc377ed1b705bcc856a628f961f1e7c8ca943e6f3727b7c179c657e227e8e852c', -- 'https://github.com/miguelmota/merkletreejs'
  '0xcc8d03e014e121d10602eeff729b755d5dc6a317df0d6302c8a9d3b5424aaba8'  -- 'https://github.com/ethereum/solc-js'
)
GROUP BY 1,2,3
ORDER BY 4 DESC, 2
""")

df_qa

,project_name,display_name,open_source_observer_slug,updated_at,defillama,github
0,0xed3d54e5394dc3ed01f15a67fa6a70e203df31c928da...,HostIT,hostit,2025-02-26 12:42:38.809,[],[https://github.com/hostit-events/hostit-offic...
1,0x808b31862cec6dccf3894bb0e76ce4ca298e4c2820e2...,Fractal Visions,fractal visions,2025-02-25 22:55:47.836,[],[https://github.com/fractal-visions/dynamic-co...
2,0xc377ed1b705bcc856a628f961f1e7c8ca943e6f3727b...,MerkleTreeJS,,2025-02-25 06:35:17.479,[],"[https://github.com/merkletreejs/merkletreejs,..."
3,0x7262ed9c020b3b41ac7ba405aab4ff37575f8b6f975e...,Sablier,,2025-02-24 14:57:03.675,[https://defillama.com/protocol/sablier],"[https://github.com/sablier-labs/v2-core, http..."
4,0xd4f0252e6ac2408099cd40d213fb6f42e8fa129b446d...,Moonwell,,2025-02-22 19:27:10.025,[https://defillama.com/protocol/moonwell],[https://github.com/moonwell-fi/moonwell-contr...
5,0xfd9f98de666c5b0a5ce96d4a4b8d4ceee9f8c2156734...,Kim Exchange,,2025-02-21 04:24:51.794,[https://defillama.com/protocol/kim-exchange],[None]
6,0x96767e87a27cdb9798f40c3a6fd78e70da611afe53a5...,Sonus Exchange,,2025-02-20 23:16:26.857,[https://defillama.com/protocol/sonus-exchange...,[https://github.com/sonusexchange/sonusexchang...
7,0x15b210abdc6acfd99e60255792b2b78714f4e8c92c4c...,MiniBridge,,2025-02-20 08:25:23.193,[],[https://github.com/defieye/mini-bridge]
8,0x9c691ba3012549259099205cc1a7ca4d09d9b7a24a1b...,Origin Protocol,,2025-02-19 17:55:57.293,[https://defillama.com/protocol/origin-protocol],[https://github.com/originprotocol/origin-dollar]
9,0xcc8d03e014e121d10602eeff729b755d5dc6a317df0d...,Solidity,,2025-02-11 09:14:16.218,[],"[https://github.com/ethereum/solidity, https:/..."


In [2]:
df_apps = client.to_pandas("""

SELECT DISTINCT
  app.project_id,
  app.project_name,
  projects.display_name,
  projects.open_source_observer_slug,
  p.project_id AS oso_project_id,
  app.round_id,
  app.created_at,
  app.updated_at
FROM stg_op_atlas_application AS app
JOIN stg_op_atlas_project AS projects
  ON app.project_id = projects.project_id
LEFT JOIN projects_v1 AS p
  ON LOWER(projects.open_source_observer_slug) = LOWER(p.project_name)
WHERE
  app.round_id IN ('7', '8')
  AND app.status = 'submitted'
  AND app.created_at >= DATE '2025-02-01'

""")

df_apps.tail()

,project_id,project_name,display_name,open_source_observer_slug,oso_project_id,round_id,created_at,updated_at
429,OVIxgHj4N8zf/sBSYEBssZf0Nyz6mCaIEJa0exm7PoM=,0x7974baaeef5e3af2e165a0911471ccddcb47d8ca7443...,Krystal,,<NA>,8,2025-02-21 07:29:34.986,2025-02-21 07:29:34.986
430,dxRQV/bqzFb4XMHere9Gwm/qK6c11ptMeFRoTDmmxWU=,0x68f3271c77c9fd815707cefabcccb85457cfc820b657...,Kropot - Where Rewards Get Real,<NA>,<NA>,8,2025-03-27 09:56:32.931,2025-03-27 09:56:32.931
431,1MvSmaSGrb163SWKc3GOGOVPaulh39cM9SVPdACdDr4=,0x17fb589e599fe05e532b90c121eccc55b1249301e783...,The Ethernaut,the ethernaut,<NA>,7,2025-02-25 19:42:12.609,2025-02-25 19:42:12.609
432,iXR35Xnzm4ioS5d0Eka/UAnUN9CHj4iasG/B8CgiWP8=,0x7539f43dc91f9c406c83a06339708e4033cdbf630916...,Seamless Protocol,<NA>,<NA>,8,2025-02-20 22:59:02.127,2025-02-20 22:59:02.127
433,lRCVGOtHuZv3FlsKZQ/VwhC1NERgrSYVPYW8sSVpfLw=,0xda59222ef8b1cd429ad08e9d1ca90f38988834e74cf3...,Fit Club,<NA>,<NA>,8,2025-02-22 09:05:44.556,2025-02-22 09:05:44.556


In [3]:
df_artifacts = client.to_pandas("""

WITH all_repository AS (
  SELECT DISTINCT
    pr.project_id,
    pr.artifact_source_id,
    pr.artifact_source,
    pr.artifact_namespace,
    pr.artifact_name,
    pr.artifact_url,
    pr.artifact_type,
    (r.artifact_id IS NULL) as needs_review
  FROM stg_op_atlas_project_repository AS pr
  LEFT JOIN int_repositories AS r ON pr.artifact_url = r.artifact_url
),
all_deployers AS (
  SELECT DISTINCT
    pd.project_id,
    pd.artifact_source_id,
    pd.artifact_source,
    pd.artifact_namespace,
    pd.artifact_name,
    pd.artifact_url,
    pd.artifact_type,
    (c.contract_address IS NULL) as needs_review
  FROM stg_op_atlas_project_deployer AS pd
  LEFT JOIN contracts_v0 AS c ON pd.artifact_url = c.root_deployer_address
),
all_defillama AS (
  SELECT DISTINCT
    pdl.project_id,
    pdl.artifact_source_id,
    pdl.artifact_source,
    pdl.artifact_namespace,
    pdl.artifact_name,
    pdl.artifact_url,
    pdl.artifact_type,
    (tvl.to_artifact_id IS NULL AND dpm.artifact_name IS NULL) as needs_review
  FROM stg_op_atlas_project_defillama AS pdl
  LEFT JOIN int_events_daily__defillama_tvl AS tvl ON pdl.artifact_name = tvl.to_artifact_name
  LEFT JOIN int_defillama_protocol_mapping AS dpm ON pdl.artifact_name = dpm.parent_protocol
),
all_artifacts AS (
  SELECT * FROM all_repository
  UNION ALL
  SELECT * FROM all_deployers
  UNION ALL
  SELECT * FROM all_defillama 
)

SELECT *
FROM all_artifacts

""")

df_artifacts.tail()

,project_id,artifact_source_id,artifact_source,artifact_namespace,artifact_name,artifact_url,artifact_type,needs_review
3254,Do7XmLDYP7t4xDWAldvqGF8KevofT/SUPeKGpg089ag=,https://defillama.com/protocol/jonascoolprotocool,DEFILLAMA,,jonascoolprotocool,https://defillama.com/protocol/jonascoolprotocool,DEFILLAMA_PROTOCOL,True
3255,5DRU+kSqJj07dgVZUng7jkEn6Pqn3SZm2yQWrZEAS8M=,https://defillama.com/protocol/aave,DEFILLAMA,,aave,https://defillama.com/protocol/aave,DEFILLAMA_PROTOCOL,False
3256,7o3rQns2QxmMIyr/Lp6EXIqBE9mvqX+1V6tQ8g1t/MM=,https://defillama.com/protocol/woofi,DEFILLAMA,,woofi,https://defillama.com/protocol/woofi,DEFILLAMA_PROTOCOL,False
3257,GUJFsIISwWw98/9CAUctXPzsi3BtbkbkmqhPn0rwhDo=,https://defillama.com/protocol/jonascoolprotocool,DEFILLAMA,,jonascoolprotocool,https://defillama.com/protocol/jonascoolprotocool,DEFILLAMA_PROTOCOL,True
3258,VsbnIqH0Z8ErW7VixsMTGcLVf45E645bcxAVcD/KZT0=,https://defillama.com/protocol/aave,DEFILLAMA,,aave,https://defillama.com/protocol/aave,DEFILLAMA_PROTOCOL,False


In [4]:
df = df_apps.merge(
    df_artifacts.pivot_table(
        index='project_id',
        columns='artifact_type',
        values='artifact_url',
        aggfunc='nunique',
        fill_value=0
    ).rename(columns={
        'DEFILLAMA_PROTOCOL': 'num_defillama_protocols',
        'DEPLOYER': 'num_deployer_addresses',
        'REPOSITORY': 'num_git_repositories'
    }).join(
        df_artifacts[df_artifacts['needs_review']]
        .groupby('project_id')['artifact_url']
        .agg(lambda x: ";".join(set(x)))
        .rename('artifacts_to_review')
    ).reset_index(),
    on='project_id'
)
df.tail()

,project_id,project_name,display_name,open_source_observer_slug,oso_project_id,round_id,created_at,updated_at,num_defillama_protocols,num_deployer_addresses,num_git_repositories,artifacts_to_review
420,OVIxgHj4N8zf/sBSYEBssZf0Nyz6mCaIEJa0exm7PoM=,0x7974baaeef5e3af2e165a0911471ccddcb47d8ca7443...,Krystal,,<NA>,8,2025-02-21 07:29:34.986,2025-02-21 07:29:34.986,1,3,2,<NA>
421,dxRQV/bqzFb4XMHere9Gwm/qK6c11ptMeFRoTDmmxWU=,0x68f3271c77c9fd815707cefabcccb85457cfc820b657...,Kropot - Where Rewards Get Real,<NA>,<NA>,8,2025-03-27 09:56:32.931,2025-03-27 09:56:32.931,0,1,1,<NA>
422,1MvSmaSGrb163SWKc3GOGOVPaulh39cM9SVPdACdDr4=,0x17fb589e599fe05e532b90c121eccc55b1249301e783...,The Ethernaut,the ethernaut,<NA>,7,2025-02-25 19:42:12.609,2025-02-25 19:42:12.609,0,0,1,<NA>
423,iXR35Xnzm4ioS5d0Eka/UAnUN9CHj4iasG/B8CgiWP8=,0x7539f43dc91f9c406c83a06339708e4033cdbf630916...,Seamless Protocol,<NA>,<NA>,8,2025-02-20 22:59:02.127,2025-02-20 22:59:02.127,0,1,1,<NA>
424,lRCVGOtHuZv3FlsKZQ/VwhC1NERgrSYVPYW8sSVpfLw=,0xda59222ef8b1cd429ad08e9d1ca90f38988834e74cf3...,Fit Club,<NA>,<NA>,8,2025-02-22 09:05:44.556,2025-02-22 09:05:44.556,0,1,0,<NA>


In [5]:
#df_artifacts[df_artifacts['project_id'] == '1umB0u6OkfQvegnnx2lkwvEISHWAtG3PmHCVXVHone8=']

In [6]:
#df_apps[df_apps['project_name'] == '0x48220ef1d103189cd918e9290db4c4b99b463ae2817fb5ef0cc54556a7961b6f']

In [7]:
#df.to_csv("S7-M2_ApplicationQA.csv", index=False)